In [ ]:
from ipywidgets import *
from IPython.display import IFrame, YouTubeVideo
from IPython.core.display import display
import subprocess, sys
import webbrowser
from ipywidgets import HTML
import base64
import csv ## https://docs.python.org/fr/3.6/library/csv.html 
import os ## Pour utiliser la methode ci-dessous Get Current Worked Directory pour lire la page Web crée dans le dossier courant.
Lieu = os.getcwd()  ## methode Get Current Worked Directory pour lire la page Web crée dans le dossier courant.

import os

import subprocess, sys

from time import gmtime, strftime
from datetime import datetime
from pytz import timezone

from reportlab.lib.enums import TA_JUSTIFY
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image

In [1]:
print("Hello word")

Hello word
